In [ ]:
from delta.tables import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType, TimestampType, FloatType
from pyspark.sql.functions import *

# Read data using user defined schema - more work, more control

In [ ]:

SALES_ORDERS_PATH="Files/bronze/sales/store_orders/2024/06/01/17/"

SALES_ORDERS_SCHEMA =[
    ('Op', StringType()),
    ('order_number', IntegerType()),
    ('customer_id', IntegerType()),
    ('product_id', IntegerType()),
    ('order_date', StringType()),    
    ('units', IntegerType()),
    ('sale_price', FloatType()),
    ('currency', StringType()),
    ('order_mode', StringType())
]

fields = [StructField(*field) for field in SALES_ORDERS_SCHEMA]
schema = StructType(fields)
df_read_data_incremental = spark.read                          \
                                .option("header", "true")      \
                                .csv(SALES_ORDERS_PATH, schema=schema)

display(df_read_data_incremental)
df_read_data_incremental.printSchema()

# Read data using spark inferred schema - less work, reasonable control

In [ ]:
df_read_data_incremental = spark.read                             \
                                .option("header", "true")         \
                                .option("inferSchema", "true")    \
                                .csv(SALES_ORDERS_PATH)

display(df_read_data_incremental)
df_read_data_incremental.printSchema()

# Perform some date standardization

In [ ]:
df_read_data_incremental = df_read_data_incremental.withColumn("order_date", to_date(df_read_data_incremental.order_date,  'MM/dd/yyyy'))
df_read_data_incremental = df_read_data_incremental.withColumn("updated_at", to_timestamp(df_read_data_incremental.updated_at,  'yyyy-MM-dd HH:mm:ss'))
display(df_read_data_incremental)
df_read_data_incremental.printSchema()

# The data merge operation in silver table

In [ ]:
def merge_to_delta(SALES_ORDERS_PATH):
    try:
        deltaTable = DeltaTable.forPath(spark, "Tables/temp_store_orders")
        if deltaTable:
            print("Delta table exists")
            df_read_data_incremental = spark.read                             \
                                            .option("header", "true")         \
                                            .option("inferSchema", "true")    \
                                            .csv(SALES_ORDERS_PATH)
            df_read_data_incremental = df_read_data_incremental.withColumn("order_date", to_date(df_read_data_incremental.order_date,  'MM/dd/yyyy'))
            df_read_data_incremental = df_read_data_incremental.withColumn("updated_at", to_timestamp(df_read_data_incremental.updated_at,  'yyyy-MM-dd HH:mm:ss'))
            display(df_read_data_incremental)
            deltaTable.alias("store_orders").merge(
            df_read_data_incremental.alias("store_orders_incremental"),
                    "store_orders.order_number = store_orders_incremental.order_number")                     \
                    .whenMatchedUpdate(set = {"order_number":     "store_orders_incremental.order_number",   \
                                              "customer_id":      "store_orders_incremental.customer_id",    \
                                              "product_id":       "store_orders_incremental.product_id",     \
                                              "order_date":       "store_orders_incremental.order_date",     \
                                              "units":            "store_orders_incremental.units",          \
                                              "sale_price":       "store_orders_incremental.sale_price",     \
                                              "currency":         "store_orders_incremental.currency",       \
                                              "order_mode":       "store_orders_incremental.order_mode",     \
                                              "updated_at":       "store_orders_incremental.updated_at"} )   \
                    .whenNotMatchedInsert(values =                                                           \
                       {                                                    
                                              "order_number":   "store_orders_incremental.order_number",     \
                                              "customer_id":      "store_orders_incremental.customer_id",    \
                                              "product_id":       "store_orders_incremental.product_id",     \
                                              "order_date":       "store_orders_incremental.order_date",     \
                                              "units":            "store_orders_incremental.units",          \
                                              "sale_price":       "store_orders_incremental.sale_price",     \
                                              "currency":         "store_orders_incremental.currency",       \
                                              "order_mode":       "store_orders_incremental.order_mode",     \
                                              "updated_at":       "store_orders_incremental.updated_at"      \
                       }                                                                                     \
                     ).execute()
    except:
        print("Delta table does not exist")
        df_read_data_full = spark.read                          \
                                 .option("header", "true")      \
                                 .option("inferSchema", "true") \
                                 .csv(SALES_ORDERS_PATH)
        
        df_read_data_full = df_read_data_full.withColumn("order_date", to_date(df_read_data_full.order_date,  'MM/dd/yyyy'))
        df_read_data_full = df_read_data_full.withColumn("updated_at", lit(current_timestamp()))
        PARTITION_COLUMN="currency"
        df_read_data_full.write.format("delta").partitionBy(PARTITION_COLUMN).saveAsTable("temp_store_orders")
        display(df_read_data_full)

In [ ]:
merge_to_delta(SALES_ORDERS_PATH)

In [ ]:
%%sql		
SELECT * FROM temp_store_orders;

# Show the structure of the delta table. Notice the partition column

In [ ]:
%%sql
DESCRIBE temp_store_orders;

In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;

In [ ]:
%%sql
SELECT * FROM temp_store_orders WHERE order_number=5; 

In [ ]:
%%sql
UPDATE temp_store_orders SET sale_price=90.50 WHERE order_number=5;

In [ ]:
%%sql
SELECT * FROM temp_store_orders WHERE order_number=5;


In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;


In [ ]:
%%sql
SELECT * FROM temp_store_orders VERSION AS OF 0 WHERE order_number=5;

In [ ]:
%%sql
DELETE FROM temp_store_orders WHERE order_number=5;
SELECT * FROM temp_store_orders WHERE order_number=5;


In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;

In [ ]:
%%sql
RESTORE TABLE temp_store_orders TO VERSION AS OF 1;
SELECT * FROM temp_store_orders WHERE order_number=5; 

In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;


In [ ]:
%%sql
SELECT count(*) FROM temp_store_orders;

In [ ]:
SALES_ORDERS_PATH="Files/bronze/sales/store_orders/2024/06/01/18/"
merge_to_delta(SALES_ORDERS_PATH)


In [ ]:
%%sql
SELECT count(*) FROM temp_store_orders;
SELECT * FROM temp_store_orders WHERE order_number IN (500, 1254, 1501, 2234, 2345);


In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;


In [ ]:
SALES_ORDERS_PATH="Files/bronze/sales/store_orders/2024/06/01/19/"
merge_to_delta(SALES_ORDERS_PATH)


In [ ]:
%%sql
SELECT * FROM temp_store_orders WHERE order_number IN (500, 1254, 1501, 2234, 2345);

In [ ]:
%%sql
DESCRIBE HISTORY temp_store_orders;

In [ ]:
FILE_PATH_WITH_NEW_SCHEMA="Files/schema_change.csv"

df_read_data_schema_change = spark.read                           \
                                .option("header", "true")         \
                                .option("inferSchema", "true")    \
                                .csv(FILE_PATH_WITH_NEW_SCHEMA)
df_read_data_schema_change = df_read_data_schema_change.withColumn("order_date", to_date(df_read_data_schema_change.order_date,  'MM/dd/yyyy'))
#df_read_data_schema_change = df_read_data_schema_change.withColumn("updated_at", to_timestamp(df_read_data_schema_change.updated_at,  'yyyy-MM-dd HH:mm:ss'))
display(df_read_data_schema_change)
df_read_data_schema_change.printSchema()

In [ ]:
deltaTable = deltaTable = DeltaTable.forPath(spark, "Tables/temp_store_orders")

In [ ]:
df_read_data_schema_change.write.format("delta").mode("append").save("Tables/temp_store_orders")

In [ ]:
df_read_data_schema_change.write.format("delta").mode("append").option("mergeSchema", "true").save("Tables/temp_store_orders")


In [ ]:
%%sql
SELECT * FROM temp_store_orders;

In [ ]:
%%sql
DROP TABLE temp_store_orders;